# Инструменты для работы с языком

... или зачем нужна предобработка.

## Задача: классификация твитов по тональности

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/ilkte35m35l38mr/negative.sql).

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [3]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

## Baseline: классификация необработанных n-граммах

### Векторизаторы

Самый простой способ извлечь фичи из текстовых данных -- векторизаторы. 

In [5]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)

In [8]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       0.77      0.76      0.76     28528
   positive       0.76      0.77      0.76     28181

avg / total       0.76      0.76      0.76     56709



Попробуем сделать то же самое для триграмм:

In [9]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       0.47      0.72      0.56     18097
   positive       0.82      0.61      0.70     38612

avg / total       0.71      0.65      0.66     56709



(как вы думаете, почему в результатах теперь такой разброс по сравнению с униграммами?)

## TF-IDF векторизация

\*тут кусок теории про тфидф\*

Сакральный смысл – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       0.74      0.77      0.75     26711
   positive       0.79      0.75      0.77     29998

avg / total       0.76      0.76      0.76     56709



## Токенизация

По дефолту векторизаторы используют свои *токенизаторы*, но можно это изменить, задав аргумент `tokenizer`.

In [14]:
from nltk.tokenize import word_tokenize

In [13]:
word_tokenize('Но не каждый хочет что-то исправлять:(')

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']

In [15]:
# TODO: 100500 токеницаторов nltk

## Стоп-слова и пунктуация

*Стоп-слова* -- это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [10]:
from nltk.corpus import stopwords

In [17]:
# TODO: remove punct and stop-words there
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

KeyboardInterrupt: 

А что, если бы их не было? Усложним себе жизнь:

In [10]:
super_significant = ['(', ')'] # добавтьте сюда эти токены

In [16]:
def remove_features(to_remove):
    pass

In [9]:
# TODO: comment
pred = ['positive' if ')' in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       1.00      0.85      0.92     32733
   positive       0.83      1.00      0.91     23976

avg / total       0.93      0.91      0.91     56709



## Словарь, закон Ципфа и закон Хипса

In [1]:
import matplotlib.pyplot as plt

In [ ]:
# картиночки и рассуждения про то, что со ременем словарь насыщается

## Лемматизация

бла бла

### [Mystem](https://tech.yandex.ru/mystem/)
Как с ним работать:
* можно скачать mystem и запускать [из терминала с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) - обертка для питона, работает медленнее, но это удобно

In [11]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

Сейчас мы запустили Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin - путь к `mystem`, если их несколько
* grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
* disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
* entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

А теперь напишите аналогичную функцию для pymorphy2:

### mystem vs. pymorphy

*мы надеемся, что вы пользуетесь лингуксом*, но mystem работает невероятно медленно под windows на больших текстах.

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом -- главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [16]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       1.00      1.00      1.00     27764
   positive       1.00      1.00      1.00     28945

avg / total       1.00      1.00      1.00     56709



Шок! Стоило оставить пунктуацию -- и все метрики равны 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?). Постройте график с весами фичей:

## Символьные n-граммы

Теперь в качестве фичей используем, например, триграммы символов:

In [15]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

   negative       0.99      1.00      1.00     27868
   positive       1.00      0.99      1.00     28841

avg / total       1.00      1.00      1.00     56709



В общем-то, теперь уже понятно почему на этих данных здесь 1. Просто знайте, что так можно.

## Fast-text